In [1]:
import mindloggerTests as mt
import numpy as np
import time
from girder_client import HttpError

### Create a new user

In [2]:
gc = mt.createGirder('localhost', 8080)
user = mt.testCreateUser(gc)
mt.authenticate(gc, user)

##### Make sure the new user has 0 applets

In [3]:
no_applets = mt.getAppletsUser(gc, user, 0)

### Add an applet and make sure it was added

In [4]:
activitySetUrl = 'https://raw.githubusercontent.com/\
ReproNim/schema-standardization/master/activity-sets/\
ema-hbn/ema-hbn_schema'

act1 = 'https://raw.githubusercontent.com/\
ReproNim/schema-standardization/master/\
activities/EmaHBNEvening/ema_evening_schema'
act2 = 'https://raw.githubusercontent.com/\
ReproNim/schema-standardization/master/\
activities/EmaHBNMorning/ema_morning_schema'

act1Item = 'https://raw.githubusercontent.com/\
ReproNim/schema-standardization/master/\
activities/EmaHBNEvening/items/good_bad_day'
act2Item = 'https://raw.githubusercontent.com/\
ReproNim/schema-standardization/master/\
activities/EmaHBNMorning/items/sleeping_aids'

In [5]:
appletObject = mt.addApplet(gc, user, activitySetUrl)

In [6]:
appletList = mt.getAppletsUser(gc, user, 1)
checkItWasAdded = mt.getAppletById(gc, user, appletObject)

### Expand and refresh the applet

In [7]:
appletsExpanded = mt.getExpandedApplets(gc, user)

In [8]:
appletRefreshed = mt.refreshApplet(gc, user, appletObject)

### Add a schedule to the applet

In [9]:
scheduleAdded = mt.addSchedule(gc, user, appletObject)

### Create a new user and invite them to the applet

In [10]:
userB = mt.testCreateUser(gc)
userBInvite = mt.inviteUserToApplet(gc, user, appletObject, userB)

##### check that manager can see invited user

In [11]:
mt.checkInvitesForUser(gc, user, appletObject, userB)

1

### Accept the applet invite

In [12]:
mt.acceptAppletInvite(gc, userB, appletObject)

1

### Invite someone who doesn't have an account yet

In [13]:
userCemail = 'randomuserc{}@test.com'.format(np.random.randint(1000000))
inviteC = mt.inviteUserToApplet(gc, user, appletObject,
                                {'email': userCemail})
appletUserTable = mt.checkAppletUserTableForUser(gc, user,
                                              appletObject,
                                              {'email': userCemail})

### Create that person's account, check they have an invite, and accept the invite

In [14]:
userC = mt.testCreateUser(gc, userCemail)
userCApplets = mt.getAppletsUser(gc, userC, 0)

# check from perspective of admin and user, if the invite exists.
mt.checkInvitesForUser(gc, user, appletObject, userC)
mt.checkForInvite(gc, userC, appletObject)

# accept
mt.acceptAppletInvite(gc, userC, appletObject)
userCApplets = mt.getAppletsUser(gc, userC, 1)

### Post Responses

In [15]:
# each user posts a response for a single item in each activity.
for u in [user, userB, userC]:
    for i in range(2):
        mt.postResponse(gc, u, act1, act1Item, appletObject)
        time.sleep(1)
        mt.postResponse(gc, u, act2, act2Item, appletObject)
        time.sleep(1)
    time.sleep(1)

### Get the last 7 days

In [16]:
last7user = mt.getLast7Days(gc, user, appletObject)
last7userB = mt.getLast7Days(gc, userB, appletObject)
last7userC = mt.getLast7Days(gc, userC, appletObject)

In [17]:
assert len(last7user['responses']) == 2, 'there should only be 2 responses'
assert len(last7userB['responses']) == 2, 'there should only be 2 responses'
assert len(last7userC['responses']) == 2, 'there should only be 2 responses'

### As a manager, see the data

In [18]:
appletData = mt.getDataForApplet(gc, user, appletObject)
assert '@' in appletData[0]['userId'], 'manager cannot see emails'

### Add a user as a reviewer

In [19]:
userCReviewer = mt.makeAReviewer(gc, user, appletObject, userC)
userCAcceptReviewer = mt.acceptReviewerInvite(gc, userC, appletObject)

### As a reviewer, see the data and make sure there aren't emails.

In [20]:
appletData = mt.getDataForApplet(gc, userC, appletObject)
assert '@' not in appletData[0]['userId'], 'reviewer can see emails'

### Make sure we don't see any data as a non-manager or non-reviewer

In [33]:
mt.testPrivacyCheck(gc, userB, appletObject)

AttributeError: module 'mindloggerTests' has no attribute 'testPrivacyCheck'

### Remove an applet without deleting data

In [ ]:
mt.removeApplet()

### Remove an applet and delete data

### deactivate an applet

### remove all users